# Module 5 Practice 1 - FHIR
In this practice exercise you'll practice accessing data from a FHIR server, storing the data in dataframes, and joining them together.

In [1]:
import sys
!{sys.executable} -m pip install fhirclient
from fhirclient import client
import pandas as pd

     |████████████████████████████████| 552kB 3.4MB/s eta 0:00:01
     |████████████████████████████████| 51kB 6.4MB/s  eta 0:00:01


Access FHIR data for patient id `22488842-c0a8-495e-b3a0-dbb05e986bda`.  Retrieve the patient's id, full name, and date of birth, and place it into a dataframe.  Make the patient id the index. Display the dataframe

In [2]:
import fhirclient.models.patient as p

settings = {
    'app_id': 'my_web_app',
    'api_base': 'https://r3.smarthealthit.org/'
}
smart = client.FHIRClient(settings=settings)

patient_id = '22488842-c0a8-495e-b3a0-dbb05e986bda'
patient = p.Patient.read(patient_id, smart.server)

name = smart.human_name(patient.name[0])
dob = patient.birthDate.isostring

patient = pd.DataFrame([[patient_id, name, dob]], columns = ['id', 'name', 'dob']).set_index('id')
display(patient)

,name,dob
id,,
22488842-c0a8-495e-b3a0-dbb05e986bda,Mrs. Chara Maggio,1965-07-17


Retrieve the list of immunizations for this patient.  Store the patient id, immunization text, the immunization code, and date of the immunizations in a dataframe.  Make the patient id the index.  Display the dataframe.
Note: access the [FHIR documentation](http://docs.smarthealthit.org/client-py/annotated.html) and search for immunization.  The pattern of use is similar for Conditions, etc.

In [3]:
import fhirclient.models.immunization as i

imms = pd.DataFrame(columns={'id': 'str', 'imm_text': 'str', 'imm_code': 'str', 'imm_date': 'date'})

search = i.Immunization.where(struct={'patient': patient_id})

immunizations = search.perform_resources(smart.server)

for imm in immunizations:
    imm_text = imm.vaccineCode.text
    imm_code = imm.vaccineCode.coding[0].code
    imm_date = imm.date.date

    imms = imms.append([{'id': patient_id,
                         'imm_text': imm_text,
                         'imm_code': imm_code,
                         'imm_date': imm_date}], ignore_index=True)

imms = imms.set_index('id')
display(imms)

,imm_text,imm_code,imm_date
id,,,
22488842-c0a8-495e-b3a0-dbb05e986bda,"Influenza, seasonal, injectable, preservative ...",140,2011-11-16 12:12:11-05:00
22488842-c0a8-495e-b3a0-dbb05e986bda,"Influenza, seasonal, injectable, preservative ...",140,2009-04-22 23:41:59-04:00
22488842-c0a8-495e-b3a0-dbb05e986bda,"Influenza, seasonal, injectable, preservative ...",140,2014-03-21 12:33:07-04:00
22488842-c0a8-495e-b3a0-dbb05e986bda,"Influenza, seasonal, injectable, preservative ...",140,2016-07-10 06:17:04-04:00
22488842-c0a8-495e-b3a0-dbb05e986bda,"Influenza, seasonal, injectable, preservative ...",140,2007-09-20 22:31:35-04:00
22488842-c0a8-495e-b3a0-dbb05e986bda,"Influenza, seasonal, injectable, preservative ...",140,2015-06-18 01:26:20-04:00
22488842-c0a8-495e-b3a0-dbb05e986bda,Td (adult) preservative free,113,2014-03-21 12:33:07-04:00


Join the patient dataframe and the immunization dataframe together on patient id and display the result

In [4]:
display(patient.join(imms))

,name,dob,imm_text,imm_code,imm_date
id,,,,,
22488842-c0a8-495e-b3a0-dbb05e986bda,Mrs. Chara Maggio,1965-07-17,"Influenza, seasonal, injectable, preservative ...",140,2011-11-16 12:12:11-05:00
22488842-c0a8-495e-b3a0-dbb05e986bda,Mrs. Chara Maggio,1965-07-17,"Influenza, seasonal, injectable, preservative ...",140,2009-04-22 23:41:59-04:00
22488842-c0a8-495e-b3a0-dbb05e986bda,Mrs. Chara Maggio,1965-07-17,"Influenza, seasonal, injectable, preservative ...",140,2014-03-21 12:33:07-04:00
22488842-c0a8-495e-b3a0-dbb05e986bda,Mrs. Chara Maggio,1965-07-17,"Influenza, seasonal, injectable, preservative ...",140,2016-07-10 06:17:04-04:00
22488842-c0a8-495e-b3a0-dbb05e986bda,Mrs. Chara Maggio,1965-07-17,"Influenza, seasonal, injectable, preservative ...",140,2007-09-20 22:31:35-04:00
22488842-c0a8-495e-b3a0-dbb05e986bda,Mrs. Chara Maggio,1965-07-17,"Influenza, seasonal, injectable, preservative ...",140,2015-06-18 01:26:20-04:00
22488842-c0a8-495e-b3a0-dbb05e986bda,Mrs. Chara Maggio,1965-07-17,Td (adult) preservative free,113,2014-03-21 12:33:07-04:00
